**Paul Lee**

In [0]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [0]:
import sys, torch, datetime, os
from pathlib import Path
sys.path.append('../modules')
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, accuracy_score, recall_score, balanced_accuracy_score, f1_score, roc_auc_score, log_loss, roc_curve
#import evaluation as e
pd.set_option('display.max_columns', 9999)

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/HealthINCITE/patient_panel/master/data/processed/test.csv')
df.head()

,person_id,myr,cad0,cad1,dv9,date,lab_1,lab_2,lab_3,ref_m,ref,class,labels,lag1_lab_1,lag1_lab_2,lag1_lab_3,sum_lab_1,sum_lab_2,sum_lab_3,win6_lab_1,win6_lab_2,win6_lab_3
0,1000,201601,1,1,0,2016-01-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
1,1000,201602,1,1,0,2016-02-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
2,1000,201603,1,1,0,2016-03-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
3,1000,201604,1,1,0,2016-04-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
4,1000,201605,1,1,0,2016-05-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0


In [5]:
def train_test_split(df, date_col, date_format, split_time):
    """
    Provide an train/test split based on a timestamp.
    df = Dataframe (Pandas dataframe).
    date_col = Date column (string).
    date_format = The date format.
    split_time = A specific place to date. (date format)
    """
    split =pd.Timestamp(split_time)
    #Let's convert this to datetime while we are at it.
    date = pd.to_datetime(df[date_col], format=date_format)
    train=df.loc[date<=split]
    test=df.loc[date>split]
    return train, test

date_col='myr'
date_format='%Y%m'
split_time=datetime.date(2016, 12,30)

train, test = train_test_split(df, date_col, date_format, split_time) 
test.head()

,person_id,myr,cad0,cad1,dv9,date,lab_1,lab_2,lab_3,ref_m,ref,class,labels,lag1_lab_1,lag1_lab_2,lag1_lab_3,sum_lab_1,sum_lab_2,sum_lab_3,win6_lab_1,win6_lab_2,win6_lab_3
12,1000,201701,1,1,0,2017-01-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
13,1000,201702,1,1,0,2017-02-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
14,1000,201703,1,1,0,2017-03-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
15,1000,201704,1,1,0,2017-04-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0
16,1000,201705,1,1,0,2017-05-01,0,0,0,0,0,h,0,0,0,0,0,0,0,0,0,0


In [6]:
#currently assumes predictions are in a dataframe. 
#With the person and category.
pred_df=pd.read_csv('https://raw.githubusercontent.com/HealthINCITE/patient_panel/master/data/predictions/test.csv')
pred_df

,person_id,ref,class
0,1000,0.0,healthy
1,1001,0.0,healthy
2,1002,0.0,healthy
3,1003,0.0,healthy
4,1004,0.7,diabetes
5,1005,0.7,diabetes
6,1006,0.8,liver
7,1007,0.8,liver
8,1008,0.9,pnemonia
9,1009,0.9,pnemonia


In [112]:
pred_file = pred_df[['person_id']]
#pred_file['ref'] = pred_file['ref'].round()
#pred_file.rename(columns={"ref": "model1"}, inplace=True)

from random import shuffle
from random import randint

a = pred_df["ref"].round()
name = "model"

for i in range(1,10):
    for j in range(1,randint(4,7)):
      shuffle(a)
      pred_file["model"+str(i)+"_Time"+str(j)] = a

pred_file

,person_id,model1_Time1,model1_Time2,model1_Time3,model2_Time1,model2_Time2,model2_Time3,model2_Time4,model2_Time5,model2_Time6,model3_Time1,model3_Time2,model3_Time3,model3_Time4,model3_Time5,model4_Time1,model4_Time2,model4_Time3,model4_Time4,model5_Time1,model5_Time2,model5_Time3,model5_Time4,model6_Time1,model6_Time2,model6_Time3,model6_Time4,model6_Time5,model7_Time1,model7_Time2,model7_Time3,model7_Time4,model7_Time5,model7_Time6,model8_Time1,model8_Time2,model8_Time3,model8_Time4,model8_Time5,model8_Time6,model9_Time1,model9_Time2,model9_Time3
0,1000,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
1,1001,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1002,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
3,1003,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1004,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
5,1005,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
6,1006,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7,1007,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
8,1008,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
9,1009,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [113]:
def score_landmarks(exp, per_col, date_col, test_file, pred_file, k, landmarks, results_file,  save=True, append=True):
    """
    exp = Experiment name.
    per_col = Person Column 
    date_col= Date Col
    df = Evaluation dataframe
    df_pred= Predictions
    k = The maximum number of recommendations of positive class.
    landmarks = Specific windows, such that [[0,3], [0,6], [0,12]] coresponds to col 0-3, 0-6, 0-12.
 
    """
    #Hard coding in file to run through the eval function

    df_test = test_file
    ###########df_test=pd.read_csv(test_file)
    df_pred = pred_file
    ###########df_pred=pd.read_csv(pred_file)
    num_targets = [per_col]
    #Loop through len of columns in given prediction file and rename to targets
    for i in range(1,len(pred_file.columns)):
      num_targets.append("target_"+pred_file.columns[i])
    df_pred.columns = num_targets
    #Loop through len of model columns and create ref_bin for eval score functions 
    for i in range(1,len(num_targets)):
      df_pred['ref_bin'+pred_file.columns[i]] = df_pred[pred_file.columns[i]].astype(int)


#This creates a wide version of the dataset with columns equal to dates and rows equal to personid
    ir=df_test.pivot_table(index=per_col, columns=date_col, values= 'ref', aggfunc='sum')
    c=ir.columns
    #Initialize a results data frame.
    results=pd.DataFrame() #final results
    row=0
    #Sort by capacity and set positive class based on top probabilities
 
    
    #Filter out people we don't have a prediction for
    ir=ir[ir.index.isin(df_pred[per_col])]
    
    # Loop through the windows
    for w in landmarks:
      for i in range(1,len(num_targets)):
        sl=slice(w[0],w[1])
        y= ir.iloc[:,sl].sum(axis=1) #take slice based on window
        #If more than 1 referral in window, recode to 1
        y[y>1]=1
        if df_pred.shape[0]!=len(y):
            #print("df with ",y, " people;",df_pred.shape[0], " predictions" )
            exit
        label=str(c[w[0]])+'-'+ str(c[w[1]-1])
        #label=c[w[0]].strftime('%Y%m')+'-'+c[w[1]-1].strftime('%Y%m')
        results.loc[row, 'experiment']=exp+" for "+pred_file.columns[i].replace('target_','')
        results.loc[row, 'date']=pd.Timestamp.now(tz=None)
        results.loc[row, 'n'] = df_pred.shape[0]
        results.loc[row, 'range']=label
        results.loc[row, 'log_loss'] = log_loss(y, df_pred[pred_file.columns[i]])
        results.loc[row, 'roc_auc_score'] = roc_auc_score(y, df_pred[pred_file.columns[i]])
        #loop through to evaluate for different K
        for lim in k: 
            results.loc[row, 'precision@'+str(lim)]=precision_score(y, df_pred[pred_file.columns[i]])
            results.loc[row, 'recall@'+str(lim)]=recall_score(y, df_pred[pred_file.columns[i]])
            results.loc[row, 'accuracy@'+str(lim)]=accuracy_score(y, df_pred[pred_file.columns[i]])
            results.loc[row, 'balanced_accuracy@'+str(lim)]=balanced_accuracy_score(y, df_pred[pred_file.columns[i]])
            results.loc[row, 'f1@'+str(lim)]=f1_score(y, df_pred[pred_file.columns[i]])
        row=row+1
    if save:
        if append and os.path.exists(results_file):
            with open(results_file, 'a') as f:
                results.to_csv(f, header=False)
        else: 
            results.to_csv(results_file, index = False)
           
    return results    

 
#define the windows.  For example [0,3] is including between 0-3 months.
exp="Ground Truth"
results_file ='results.csv'

test_file = test
########test_file='test_2016.csv'
pred_file = pred_file
########pred_file='gt_2016.csv'
 
k=[100] #k is used the set the conversion from prob to class prediction
per_col='person_id'
 
landmarks= [[0,3], [0,6], [0,12]]
#Score windows
results=score_landmarks(exp, per_col, date_col, test_file, pred_file, k, landmarks, results_file,  save=True, append=True)
results

#Yeah, if the time was in the column name I think that would be a flexible design.
#Add train time period cutoff to model names which will be evaluated over the landmarks/windows


,experiment,date,n,range,log_loss,roc_auc_score,precision@100,recall@100,accuracy@100,balanced_accuracy@100,f1@100
0,Ground Truth for model1_Time1,2020-02-26 22:45:56.032197888,12.0,201701-201703,11.513192,0.800000,0.333333,1.000000,0.666667,0.800000,0.500000
1,Ground Truth for model1_Time2,2020-02-26 22:45:56.060948992,12.0,201701-201703,17.269721,0.500000,0.166667,0.500000,0.500000,0.500000,0.250000
2,Ground Truth for model1_Time3,2020-02-26 22:45:56.077890048,12.0,201701-201703,23.026251,0.200000,0.000000,0.000000,0.333333,0.200000,0.000000
3,Ground Truth for model2_Time1,2020-02-26 22:45:56.093884928,12.0,201701-201703,11.513192,0.800000,0.333333,1.000000,0.666667,0.800000,0.500000
4,Ground Truth for model2_Time2,2020-02-26 22:45:56.110301952,12.0,201701-201703,17.269721,0.500000,0.166667,0.500000,0.500000,0.500000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...
121,Ground Truth for model8_Time5,2020-02-26 22:45:58.025441024,12.0,201701-201712,17.269588,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
122,Ground Truth for model8_Time6,2020-02-26 22:45:58.042350080,12.0,201701-201712,23.026117,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
123,Ground Truth for model9_Time1,2020-02-26 22:45:58.058693120,12.0,201701-201712,11.513059,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667
124,Ground Truth for model9_Time2,2020-02-26 22:45:58.075558912,12.0,201701-201712,28.782647,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667


In [0]:
#Code for testing strucuture of pred_file ***IGNORE

#pred_file.columns
#x = ['person_id']
#for i in range(1,len(pred_file.columns)):
#  x.append("target"+str(i))
#pred_file.columns = x
#pred_file.columns

#for i in range(1,len(pred_file.columns)+1):
#  pred_file['ref_bin'+str(i)] = pred_file['target'+str(i)].astype(int)
#pred_file

#df_test = test_file
###########df_test=pd.read_csv(test_file)
#df_pred = pred_file
###########df_pred=pd.read_csv(pred_file)
#num_targets = [per_col]
#Loop through len of columns in given prediction file and rename to targets
#for i in range(1,len(pred_file.columns)):
#    num_targets.append("target_"+pred_file.columns[i])
#df_pred.columns = num_targets


#Loop through len of model columns and create ref_bin for eval score functions 
#for i in range(1,len(num_targets)):
#    df_pred['ref_bin'+num_targets[i]] = df_pred[num_targets[i]].astype(int)

